In [1]:
import os
import pandas as pd

# 定义CSV文件所在目录
directory = './nasdaq/csv'

# 初始化一个空的列表用于存放结果
results = []

# 遍历目录下的所有文件
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # 构造文件路径
        file_path = os.path.join(directory, filename)
        
        # 读取CSV文件，处理错误行
        try:
            df = pd.read_csv(file_path, on_bad_lines='skip')
            
            # 转换 'Date' 列为 datetime 类型
            df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
            
            # 获取第一个和最后一个时间戳
            first_timestamp = df['Date'].min()
            last_timestamp = df['Date'].max()
            
            # 将结果添加到 results 列表中
            results.append({
                'File Name': os.path.splitext(filename)[0],
                'First Timestamp': first_timestamp,
                'Last Timestamp': last_timestamp
            })
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

# 使用 pd.DataFrame 将结果转换为 DataFrame
timestamps = pd.DataFrame(results)

# 输出结果
print(timestamps)

Error processing file LRCX.csv: time data '18-1218-12-1991' does not match format '%d-%m-%Y' (match)
     File Name First Timestamp Last Timestamp
0         CSCO      1990-02-16     2022-12-12
1         BIOS      2022-01-28     2022-12-12
2         CSBK      2004-03-04     2022-12-12
3         SBGI      1995-06-07     2022-12-12
4         UFCS      1980-03-18     2022-12-12
...        ...             ...            ...
1558       EDF      2010-12-27     2022-12-12
1559      SPXX      2005-11-23     2022-12-12
1560      NVCN      2007-02-05     2022-12-12
1561      ANTH      2010-03-01     2022-12-12
1562      BWEN      2005-11-10     2022-12-12

[1563 rows x 3 columns]


In [4]:

result_df = timestamps[timestamps['File Name'] == 'TSM']

# 显示结果
print(result_df)

Empty DataFrame
Columns: [File Name, First Timestamp, Last Timestamp]
Index: []


In [11]:
import pandas as pd

goog_df = pd.read_csv("./forbes2000/csv/GOOG.csv")

msft_df = pd.read_csv("./forbes2000/csv/MSFT.csv")

goog_df.head(5)

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,19-08-2004,2.390042,2.490664,897427216,2.591785,2.499133,2.499133
1,20-08-2004,2.503118,2.515820,458857488,2.716817,2.697639,2.697639
2,23-08-2004,2.716070,2.758411,366857939,2.826406,2.724787,2.724787
3,24-08-2004,2.579581,2.770615,306396159,2.779581,2.611960,2.611960
4,25-08-2004,2.587302,2.614201,184645512,2.689918,2.640104,2.640104


In [28]:
# 确保日期列格式一致
goog_df['Date'] = pd.to_datetime(goog_df['Date'])
msft_df['Date'] = pd.to_datetime(msft_df['Date'])

# 添加类型标记
goog_df['Type'] = 'GOOG'
msft_df['Type'] = 'MSFT'

# 合并两个 DataFrame
merged_df = pd.concat([goog_df, msft_df], ignore_index=True)

# 按照日期排序
sorted_df = merged_df.sort_values(by='Date').reset_index(drop=True)

# 输出排序后的数据
print(sorted_df)

            Date         Low        Open      Volume        High       Close  \
0     1986-03-13    0.088542    0.088542  1031788800    0.101563    0.097222   
1     1986-03-14    0.097222    0.097222   308160000    0.102431    0.100694   
2     1986-03-17    0.100694    0.100694   133171200    0.103299    0.102431   
3     1986-03-18    0.098958    0.102431    67766400    0.103299    0.099826   
4     1986-03-19    0.097222    0.099826    47894400    0.100694    0.098090   
...          ...         ...         ...         ...         ...         ...   
13871 2022-12-08   93.800003   95.690002    25593200   95.870003   93.949997   
13872 2022-12-09   93.019997   93.900002    21873700   94.489998   93.070000   
13873 2022-12-09  244.160004  244.699997    20607100  248.309998  245.419998   
13874 2022-12-12   91.900002   93.089996     7849646   93.874496   92.464996   
13875 2022-12-12  247.179993  247.445007    10000660  252.151596  250.559998   

       Adjusted Close  Type  
0        

In [46]:
# 保存到 CSV
sorted_df.to_csv('GOOG_MSFT.csv', index=False)

In [17]:
def find_increase(df, days, percent):
    """
    Find instances where the 'Close' price increases by a given percentage within a specified number of days.

    Parameters:
        df (DataFrame): Input DataFrame with at least 'Date' and 'Close' columns.
        days (int): Number of days to look ahead.
        percent (float): Percentage increase to check for (e.g., 0.2 for 20%).

    Returns:
        DataFrame: Results with 'Start Date', 'Start Price', 'End Date', 'End Price', and 'Increase'.
    """
    # Ensure 'Date' is in datetime format
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
    
    results = []
    for i in range(len(df)):
        start_date = df.iloc[i]['Date']
        start_price = df.iloc[i]['Close']
        end_date_limit = start_date + pd.Timedelta(days=days)
        
        # Get subset of rows within the specified time window
        subset = df[(df['Date'] > start_date) & (df['Date'] <= end_date_limit)]
        
        # Check if any 'Close' price increases by the given percentage within the window
        for _, row in subset.iterrows():
            if (row['Close'] - start_price) / start_price >= percent:
                results.append({
                    'Start Date': start_date,
                    'Start Price': start_price,
                    'End Date': row['Date'],
                    'End Price': row['Close'],
                    'Increase': (row['Close'] - start_price) / start_price
                })
    
    return pd.DataFrame(results)

In [47]:
result_goog = find_increase(goog_df, 20, 0.2)
result_goog

,Start Date,Start Price,End Date,End Price,Increase
0,2004-09-03,2.490913,2004-09-23,3.009221,0.208080
1,2004-09-08,2.547950,2004-09-28,3.159657,0.240078
2,2004-09-09,2.548199,2004-09-28,3.159657,0.239957
3,2004-09-09,2.548199,2004-09-29,3.264763,0.281204
4,2004-09-10,2.623417,2004-09-28,3.159657,0.204405
...,...,...,...,...,...
310,2015-07-13,27.327499,2015-07-17,33.646500,0.231232
311,2015-07-13,27.327499,2015-07-20,33.151001,0.213100
312,2015-07-13,27.327499,2015-07-21,33.115002,0.211783
313,2015-07-13,27.327499,2015-07-22,33.105000,0.211417


In [21]:
result_msft = find_increase(msft_df, 10, 0.2)
result_msft

,Start Date,Start Price,End Date,End Price,Increase
0,1986-09-30,0.098090,1986-10-10,0.119792,0.221246
1,1986-10-01,0.097222,1986-10-10,0.119792,0.232149
2,1986-10-02,0.097222,1986-10-10,0.119792,0.232149
3,1986-10-03,0.098090,1986-10-10,0.119792,0.221246
4,1986-11-18,0.140625,1986-11-24,0.175347,0.246912
...,...,...,...,...,...
86,2001-01-16,26.281250,2001-01-26,32.000000,0.217598
87,2001-01-17,26.468750,2001-01-26,32.000000,0.208973
88,2002-10-09,21.995001,2002-10-18,26.575001,0.208229
89,2007-10-22,30.510000,2007-10-31,36.810001,0.206490


In [68]:
import pandas as pd
from itertools import combinations

def find_pattern(sorted_df, n, percent):
    """
    Identify patterns A B+ C where:
        - A and C are events from 'msft'.
        - B+ is one or more events from 'goog'.
        - C's Close is higher than A's Close by a given percentage within n days.
        - B+ has a subsequence of Close values increasing for at least n/2 days.
    
    Parameters:
        sorted_df (DataFrame): Combined sorted data with a 'Source' column ('msft' or 'goog').
        n (int): Number of days for the pattern window.
        percent (float): Percentage increase for C relative to A.
    
    Returns:
        DataFrame: Rows matching the pattern with Start Date, End Date, and details.
    """
    results = []
    sorted_df['Date'] = pd.to_datetime(sorted_df['Date'], format='%d-%m-%Y')

    # Separate data by source
    msft_df = sorted_df[sorted_df['Type'] == 'MSFT']
    goog_df = sorted_df[sorted_df['Type'] == 'GOOG']

    for i, row_a in msft_df.iterrows():
        start_date = row_a['Date']
        start_price = row_a['Close']
        end_date_limit = start_date + pd.Timedelta(days=n)

        # Find all potential C candidates
        potential_c = msft_df[(msft_df['Date'] > start_date) & (msft_df['Date'] <= end_date_limit)]
        for _, row_c in potential_c.iterrows():
            if (row_c['Close'] - start_price) / start_price >= percent:
                # Check B+ conditions
                goog_subset = goog_df[(goog_df['Date'] > start_date) & (goog_df['Date'] <= row_c['Date'])]
                increasing_subsequences = find_increasing_combinations(goog_subset['Close'].values, n // 2)
                if increasing_subsequences:
                    results.append({
                        'Start Date (A)': start_date,
                        'Start Price (A)': start_price,
                        'End Date (C)': row_c['Date'],
                        'End Price (C)': row_c['Close'],
                        'Increase (%)': (row_c['Close'] - start_price) / start_price,
                        'B+ Increasing Subsequences': increasing_subsequences,
                        'B+ Events': goog_subset.to_dict('records')
                    })
    
    return pd.DataFrame(results)


def find_increasing_combinations(arr, min_length):
    """
    Find all increasing subsequences of at least min_length using combination method.
    
    Parameters:
        arr (list): The list of numbers.
        min_length (int): Minimum required length of the increasing subsequence.
    
    Returns:
        list: All increasing subsequences of at least min_length.
    """
    if len(arr) < min_length:
        return []

    subsequences = []
    # Generate all combinations of length >= min_length
    for length in range(min_length, len(arr) + 1):
        for combination in combinations(arr, length):
            if all(combination[i] < combination[i + 1] for i in range(len(combination) - 1)):
                subsequences.append(combination)
    
    return subsequences

# 示例调用
# sorted_df = pd.DataFrame(...)  # 示例数据
results = find_pattern(sorted_df, n=20, percent=0.2)

print(results)

   Start Date (A)  Start Price (A) End Date (C)  End Price (C)  Increase (%)  \
0      2007-10-11        29.910000   2007-10-31      36.810001      0.230692   
1      2007-10-12        30.170000   2007-10-31      36.810001      0.220086   
2      2007-10-12        30.170000   2007-11-01      37.060001      0.228373   
3      2007-10-15        30.040001   2007-10-31      36.810001      0.225366   
4      2007-10-15        30.040001   2007-11-01      37.060001      0.233688   
5      2007-10-15        30.040001   2007-11-02      37.060001      0.233688   
6      2007-10-16        30.320000   2007-11-02      37.060001      0.222296   
7      2007-10-16        30.320000   2007-11-05      36.730000      0.211412   
8      2007-10-17        31.080000   2007-11-05      36.730000      0.181789   
9      2007-10-17        31.080000   2007-11-06      36.410000      0.171493   
10     2007-10-18        31.160000   2007-11-06      36.410000      0.168485   
11     2013-04-16        28.969999   201

In [66]:
# 访问 `B+ Increasing Subsequences`
increasing_subsequences = results.loc[0, "B+ Events"]
print("B+ Increasing Subsequences:", increasing_subsequences)

B+ Increasing Subsequences: [{'Date': Timestamp('2007-10-09 00:00:00'), 'Low': 15.152953147888184, 'Open': 15.32032585144043, 'Volume': 352026555, 'High': 15.536266326904297, 'Close': 15.32206916809082, 'Adjusted Close': 15.32206916809082, 'Type': 'GOOG'}, {'Date': Timestamp('2007-10-10 00:00:00'), 'Low': 15.362418174743652, 'Open': 15.475992202758787, 'Volume': 216231462, 'High': 15.583588600158691, 'Close': 15.576366424560549, 'Adjusted Close': 15.576366424560549, 'Type': 'GOOG'}, {'Date': Timestamp('2007-10-11 00:00:00'), 'Low': 15.168146133422852, 'Open': 15.781846046447754, 'Volume': 473729022, 'High': 15.975370407104492, 'Close': 15.491931915283203, 'Adjusted Close': 15.491931915283203, 'Type': 'GOOG'}, {'Date': Timestamp('2007-10-12 00:00:00'), 'Low': 15.398283004760742, 'Open': 15.541248321533203, 'Volume': 273971076, 'High': 15.90040111541748, 'Close': 15.875246047973633, 'Adjusted Close': 15.875246047973633, 'Type': 'GOOG'}, {'Date': Timestamp('2007-10-15 00:00:00'), 'Low': 1

In [23]:
!pip install ace_tools


[notice] A new release of pip is available: 23.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [56]:
# 筛选条件：Type = "GOOG" 且日期在 2020-03-23 和 2020-04-11 之间
start_date = pd.to_datetime("2020-03-22")
end_date = pd.to_datetime("2020-04-10")
filtered_df = sorted_df[(sorted_df["Type"] == "GOOG") & (sorted_df["Date"] >= start_date) & (sorted_df["Date"] <= end_date)]
filtered_df

,Date,Low,Open,Volume,High,Close,Adjusted Close,Type
12500,2020-03-23,50.676800,53.066002,80882000,53.566002,52.831001,52.831001,GOOG
12502,2020-03-24,54.530998,55.188499,66890000,56.750000,56.723000,56.723000,GOOG
12504,2020-03-25,54.300499,56.323502,81630000,57.445000,55.124500,55.124500,GOOG
12507,2020-03-26,54.676498,55.590000,71434000,58.498501,58.087502,58.087502,GOOG
12509,2020-03-27,55.295502,56.283501,64170000,57.533501,55.535500,55.535500,GOOG
12511,2020-03-30,54.824001,56.251999,51482000,57.581501,57.341000,57.341000,GOOG
12512,2020-03-31,56.907001,57.365002,49728000,58.765499,58.140499,58.140499,GOOG
12514,2020-04-01,54.872501,56.099998,46884000,56.484501,55.280998,55.280998,GOOG
12517,2020-04-02,54.820000,54.912998,39298000,56.342999,56.042000,56.042000,GOOG
12519,2020-04-03,53.990501,55.950748,46268000,56.176998,54.894001,54.894001,GOOG


In [60]:
import re
# 解析 flink_result
def parse_flink_result(flink_result):
    # 使用正则解析 a_date, c_date 和 b_date
    a_date = re.search(r"a_date: ([\d-]+\s[\d:]+)", flink_result).group(1)
    c_date = re.search(r"c_date: ([\d-]+\s[\d:]+)", flink_result).group(1)
    b_dates = re.findall(r"b_date: ([\d-]+\s[\d:]+)|, ([\d-]+\s[\d:]+)", flink_result)
    b_dates = [d[0] if d[0] else d[1] for d in b_dates]
    return {"a_date": a_date, "c_date": c_date, "b_date": b_dates}

# 解析结果
parsed_result = parse_flink_result(flink_result)

# 修复后的提取函数
def extract_pattern_data_date_only(pattern, df):
    a_date = pd.to_datetime(pattern["a_date"]).strftime('%Y-%m-%d')
    c_date = pd.to_datetime(pattern["c_date"]).strftime('%Y-%m-%d')
    b_dates = [pd.to_datetime(date).strftime('%Y-%m-%d') for date in pattern["b_date"]]
    
    a_data = df[(df["Date"] == a_date) & (df["Type"] == "MSFT")]
    c_data = df[(df["Date"] == c_date) & (df["Type"] == "MSFT")]
    b_data = df[(df["Date"].isin(b_dates)) & (df["Type"] == "GOOG")]
    
    return pd.concat([a_data, c_data, b_data])

# 调用函数
fr = extract_pattern_data_date_only(parsed_result, sorted_df)
fr

,Date,Low,Open,Volume,High,Close,Adjusted Close,Type
12501,2020-03-23,132.520004,137.009995,78975200,140.570007,135.979996,132.558426,MSFT
12527,2020-04-09,163.330002,166.360001,51385100,167.369995,165.139999,160.984695,MSFT
12502,2020-03-24,54.530998,55.188499,66890000,56.750000,56.723000,56.723000,GOOG
12509,2020-03-27,55.295502,56.283501,64170000,57.533501,55.535500,55.535500,GOOG
12511,2020-03-30,54.824001,56.251999,51482000,57.581501,57.341000,57.341000,GOOG
12512,2020-03-31,56.907001,57.365002,49728000,58.765499,58.140499,58.140499,GOOG
12514,2020-04-01,54.872501,56.099998,46884000,56.484501,55.280998,55.280998,GOOG
12517,2020-04-02,54.820000,54.912998,39298000,56.342999,56.042000,56.042000,GOOG
12520,2020-04-06,56.547001,56.900002,53294000,59.733002,59.346001,59.346001,GOOG
12523,2020-04-07,59.111500,61.049999,47746000,61.250000,59.325500,59.325500,GOOG
